In [ ]:
import manganite
%load_ext manganite

# Sudoku Solver

## Sudoku Ruleset
Sudoku is a classic number puzzle game played on a 9x9 grid. The objective is to fill each row, column, and 3x3 subgrid with the numbers 1 to 9, ensuring that no number is repeated in any row, column, or subgrid. For more details on Sudoku rules, you can refer to the [Wikipedia page](https://en.wikipedia.org/wiki/Sudoku).

## How to Use
1. **Input Grid**: On the left side of the web app, you will find a blank 9x9 grid representing the Sudoku puzzle. Empty cells are denoted by 0, indicating spaces to be filled.

2. **User Input**: You can input your known numbers into the grid by replacing the 0s with the correct digits. These represent the initial clues for the Sudoku puzzle.

3. **Optimize Button**: Once you have entered your initial numbers, click the "Optimize" button. This will trigger the solver to use the SCX Optimizer package to calculate a possible solution for the Sudoku puzzle. (you have to install scx using pip)

4. **View Solution**: The solved Sudoku grid will be displayed on the right side of the web app, showing the optimized arrangement of numbers based on your initial input.

In [ ]:
from scx.optimize import Model
import pandas
import numpy

In [ ]:
%%mnn widget --type table --var input_matrix --tab "Sudoku Solver" --header "Inputs" --position 0 0 3
input_matrix = pandas.DataFrame(numpy.zeros((9, 9), dtype=int))

In [ ]:
%%mnn execute --on button "Solve" --returns data

# Create variables
data = [{
        'row':row+1,
        'col':col+1,
        'box':(row//3)*3+(col//3)+1,
        'value':value+1,
        'use':Model.variable(name=f"sq_{row+1}_{col+1}_{value+1}", cat="Binary")
    } for row in range(9) for col in range(9) for value in range(9)]

constraints = []
for row in range(9):
    for col in range(9):
        value = input_matrix[row][col]
        if value != 0:
            constraints.append({
                'row':row+1,
                'col':col+1,
                'value':value
            })
            
# Initialize the my_model

my_model = Model(name="Sudoku", sense=None)

# Add the Objective Fn
## No is objective needed. We just need to solve for the constraints.

# Add Constraints

## Ensure that each square gets assigned exactly one value
for row_i in range(1,10):
    for col_j in range(1,10):
        my_model.add_constraint(
            Model.sum([d['use'] for d in data if d['row']==row_i and d['col']==col_j])==1
        )

## Add in constraints such that each row, col and box will only have a value once
for key_name in ['row','col','box']:
    for key_value in range(1,10):
        for data_value in range(1,10):
            my_model.add_constraint(
                Model.sum([d['use'] for d in data if d[key_name]==key_value and d['value']==data_value])==1
            )

# Add in constraints such that each square must match the provided inputs
for j in constraints:
    my_model.add_constraint(
        Model.sum([d['use'] for d in data if d['row']==j['row'] and d['col']==j['col'] and d['value']==j['value']])==1
    )


# Solve the my_model
my_model.solve()


In [ ]:
%%mnn widget --type table --var matrix --tab "Sudoku Solver" --header "Outputs" --position 0 3 3
matrix = pandas.DataFrame(numpy.zeros((9, 9), dtype=int))
used = [i for i in data if (i['use'].value())==1]
for item in used:
    # Assign the value to the matrix using a function
    matrix.at[item['row']-1, item['col']-1] = item['value']

matrix = pandas.DataFrame(matrix).T
print("Solved Sudoku:")
print(matrix)
